In [97]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/raw/raw-data.csv")

cols_keep = ["reviewText", "reviewScoreOverall"]
df = df[cols_keep].copy()

print(df.shape)
df.head(3)


(379718, 2)


/tmp/ipykernel_55042/2207392812.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/raw-data.csv")


,reviewText,reviewScoreOverall
0,Heerlijk eten en leuke sfeer. Veel keuze bij h...,9
1,Met z'n vieren dit restaurant bezocht. We hadd...,10
2,Snel gegeten tijdens een plotselinge bezoek na...,7.5


In [98]:
df["reviewScoreOverall"] = pd.to_numeric(df["reviewScoreOverall"], errors="coerce")

df = df.dropna(subset=["reviewText", "reviewScoreOverall"]).copy()

df["reviewText"] = df["reviewText"].astype(str).str.strip()
df = df[df["reviewText"].str.len() > 5]  # minimal length filter


In [99]:
stars = df["reviewScoreOverall"] / 2.0

df["ratingRounded"] = np.round(stars * 2) / 2
df["ratingRounded"] = df["ratingRounded"].clip(0, 5)

In [100]:
ok = ((df["ratingRounded"] * 2) % 1 == 0).all()
print("Alle ratings in 0,5-stappen:", ok)
print("Unieke ratings:", sorted(df["ratingRounded"].unique()))

Alle ratings in 0,5-stappen: True
Unieke ratings: [np.float64(0.5), np.float64(1.0), np.float64(1.5), np.float64(2.0), np.float64(2.5), np.float64(3.0), np.float64(3.5), np.float64(4.0), np.float64(4.5), np.float64(5.0)]


In [101]:
def clean_text(txt: str) -> str:
    t = txt.lower()

    t = re.sub(r"http\S+|www\S+", "", t)
    t = re.sub(r"<.*?>", "", t)
    t = re.sub(r"[^a-zA-ZÀ-ÿ0-9.,!?%€$ ]", " ", t)

    t = re.sub(r"\b(\d+(\.\d+)?\s*(van\s+de\s+)?sterren?)\b", "", t)
    t = re.sub(r"\b(geef|gaf|geven|score|waardering|rating)\b", "", t)
    t = re.sub(r"\b(vijf|vier|drie|twee|één|een|1|2|3|4|5)\s*sterren?\b", "", t)

    # reduceer spaties
    t = re.sub(r"\s+", " ", t).strip()
    return t

df["reviewText_clean"] = df["reviewText"].apply(clean_text)


In [102]:
df = df.drop_duplicates(subset=["reviewText_clean"])
df = df[df["reviewText_clean"].str.len() > 10]


In [103]:
df_final = df[["reviewText_clean", "ratingRounded"]].copy()
df_final.to_csv("../data/processed/processed-data.csv", index=False)
print("Clean dataset saved:", df_final.shape)

Clean dataset saved: (334305, 2)
